##### MNST 예제

MNST 문제는 분류 문제이다. 분류 문제에서 각 범주(Category)를 클래스(Class) 라고 부르고, 데이터 포인트는 샘플(Sample)이라고 불리며, 샘플의 클래스를 레이블(Label)이라고 한다.<br>

즉 MNIST는 총 10개의 범주가 있고, 10개의 클래스로 나눌 수 있다.

keras에는 이미 mnist 데이터가 포함되어 있어, 다음과 같이 데이터를 가져올 수 있다.

<image src="./images/MnistExamples.png"><image>

In [1]:
# 케라스에서 MNIST 데이터셋 가져오기
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

각 데이터는 numpy 배열 형태로 데이터를 지니고 있다.<br>

이번에는 신경망을 만들며, 신경망을 만드는 코드는 다음과 같다.

In [2]:
# 신경망 구조
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28*28,)))
network.add(layers.Dense(10, activation='softmax'))

위 코드에서 각 층은 relu 층과 softmax 층이다.<br>
이렇게 신경망에 층을 추가하여, 의미있는 표현을 점진적으로 추출하는 것이다.<br>
마지막의 소프트맥스 층의 경우는 10개의 확률 점수가 들어 있는 배열을 반환한다.

신경망이 훈련 준비를 마치기 이해서는 컴파일 단계에 포함될 3가지가 더 필요하다.
- 손실함수
- 옵티마이저
- 훈련과 테스트 과정을 모니터링할 지표

In [3]:
# 컴파일 단계
network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

또한 현재는 [0, 255] 사이의 값인 uint8타입으로 데이터를이 있으므로 변환해준다.

In [4]:
# 이미지 데이터 준비하기
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

다음으로 레이블을 범주형으로 인코딩해야 한다.

In [5]:
# 레이블 준비하기
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

이렇게 변환하면 훈련하고, 결과를 얻으면 된다.

In [6]:
# 훈련 실행과 결과
network.fit(train_images, train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 [==============================] - 3s 6ms/step - loss: 0.2561 - accuracy: 0.9253
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1020 - accuracy: 0.9694
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0679 - accuracy: 0.9797
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0484 - accuracy: 0.9855
Epoch 5/5
313/313 [==============================] - 0s 949us/step - loss: 0.0769 - accuracy: 0.9777
test_acc: 0.9776999950408936


훈현하는 동안 2개의 정보가 출력된다. 훈련 데이터에 대한 네트워크의 손실과 정확도이다.<br>
훈련시에 정확도는 0.9894까지 증가함을 알 수 있다. 그리고 테스트의 정확도는 0.9761이 나오는 것을 확인할 수 있다.<br>
이런 정확도의 차이는 과대적합(Overfitting) 때문이다.

##### 신경망을 위한 데이터 표현

신경망에서 가장 중요한 것은 데이터에 대한 이해이다. 신경망에서 가장 기초적인 데이터는 텐서(tensor)라고 부르는 다차원 배열 데이터에서 시작한다.<br>

텐서는 데이터를 위한 컨테이너이다. 텐서는 임의의 차원 개수를 가지는 행렬, 벡터에 대한 표현이라고 생각하면 된다.<br>

차원(dimension) 대신 축(axis)이라는 표현을 쓰기도 한다.

##### 스칼라(0D tensor)
하나의 숫자만 담고 있는 텐서를 스칼라(scaler) 라고 부른다. numpy에서는 float32나 float64 데이터 하나가 스칼라 텐서이다.<br>
`ndim` 속성으로 축 개수를 확인할 수 있고, 텐서의 축 개수를 랭크(rank)라고 부른다.

##### 벡터(1D tensor)
숫자의 배열을 벡터(Vector)또는 1D 텐서라고 부른다.

##### 행렬(2D tensor)
벡터의 배열을 행렬(matrix) 또는 2D 텐서라고 부른다.

##### 3D 텐서와 고차원 텐서
행렬의 배열을 3D 텐서라고 부른다. 이렇게 계속 배열을 만들어가며 텐서를 만든다. 보통은 0D에서 4D를 다루며, 동영상의 경우 5D 텐서까지 가기도 한다.

##### 핵심 속성
- 축의 개수(rank): `ndim`
- 크기(shape): `shape` 텐서의 각 축을 따라 얼마나 많은 차원이 있는지 나타내는 파이썬의 튜플 2D는 (3,5)와 같이 나타낼 수 있다.
- 데이터 타입: `dtype` 텐서에 포함된 데이터 타입 사전에 할당되어 연속된 메모리에 저장되어야 하여 가변 길이 문자열을 지원하지 않음

<span style="color: gray">일단 딥러닝에 사용되는 데이터모양 텐서(tensor)를 이해해야 할 것 같다.</span>